<a href="https://colab.research.google.com/github/LeeSeungwon89/Machine-learning_Theory/blob/master/CHAPTER4%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%91%9C%ED%98%84%EA%B3%BC%20%ED%8A%B9%EC%84%B1%20%EA%B3%B5%ED%95%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 들어가며

2차원 실수형 배열을 가진 데이터로서 각 열이 샘플에 대해 설명하면 연속형 특성(continuous feature)이라고 가정했습니다. 하지만 많은 애플리케이션에서는 이렇게 데이터가 수집되지 않습니다. 일반적인 특성의 전형적인 형태는 숫자 값이 아닌 범주형 특성(categorical feature)(=이산형 특성(discrete feature))입니다.

연속형 특성과 범주형 특성은 회귀와 분류의 차이와 비슷합니다. 다만 출력이 아닌 입력에 대한 것이란 점에서 차이가 있습니다. 앞서 살폈던 연속형 특성의 예는 픽셀 밝기, 붓꽃 측정값입니다. 범주형 특성의 예는 제품 브랜드, 색상, 판매 분류(책, 옷, 하드웨어) 등입니다. 이런 특성들은 모두 상품을 묘사하는 속성이므로 연속된 값이 아닙니다. 제품과 제품 간에는 중간값이 없고 순서도 없습니다.

하지만 데이터의 특성이 어떤 형태를 취하는지보다 중요한 것은 데이터를 표현하는 방식입니다. 이 방식에 따라 머신러닝 모델 성능에 주는 영향이 큽니다. 데이터의 스케일을 조정하는 것이 매우 중요하다는 점을 강조했습니다. 예를 들면 측정치의 스케일을 조정하지 않았을 때 센티미터, 인치의 기준에 따라 큰 차이가 생깁니다. 스케일 조정뿐만 아니라 특성의 상호작용(특성 간의 곱)이나 일반적인 다항식을 추가 특성으로 넣는 것도 방법입니다.

특정 애플리케이션에 가장 적합한 데이터 표현을 찾는 것을 '특성 공학(feature engineering)'이라고 합니다. 올바른 데이터 표현을 찾는다면 지도 학습 모델에서 적절한 매개변수를 선택하는 것보다 모델 성능을 더 크게 제고할 수 있습니다.

# 4.1 범주형 변수

예제에 사용할 데이터는 [1994년 인구 조사 데이터베이스에서 추출한 미국 성인의 소득 데이터셋](https://github.com/rickiepark/introduction_to_ml_with_python/blob/master/data/adult.data)입니다. 다운로드 받아서 직접 실습하시기 바랍니다. 이 데이터셋을 사용해서 특정 근로자의 수입이 50,000달러를 초과하는지 이하인지를 예측하고자 합니다.

In [10]:
pip install mglearn

In [11]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
%matplotlib inline

In [12]:
# 데이터셋 10개를 부르기 위한 코드입니다.
# 코드는 뒤에서 다시 설명합니다.
import os

data = pd.read_csv(os.path.join(mglearn.datasets.DATA_PATH, 'adult.data'),
                   header=None, index_col=False,
                   names=['age', 'workclass', 'fnlwgt', 'education',
                          'education-num', 'marital-status', 'occupation',
                          'relationship', 'race', 'gender', 'capital-gain',
                          'capital-loss', 'hours-per-week', 'native-country',
                          'income'])
data = data[['age', 'workclass', 'education', 'gender', 'hours-per-week',
             'occupation', 'income']]
display(data.head(11))

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K
5,37,Private,Masters,Female,40,Exec-managerial,<=50K
6,49,Private,9th,Female,16,Other-service,<=50K
7,52,Self-emp-not-inc,HS-grad,Male,45,Exec-managerial,>50K
8,31,Private,Masters,Female,50,Prof-specialty,>50K
9,42,Private,Bachelors,Male,40,Exec-managerial,>50K


데이터셋에는 여러 특성이 있습니다. 근로자 나이(age), 고용형태(workclass), 교육수준(education), 성별(gender), 주당 근로시간(hours-per-week), 직업(occupation) 등입니다.

이 예측 작업은 소득(income)이 <=50k, >50k라는 두 클래스를 가진 분류 문제입니다. 정확한 소득을 예측하고자 한다면 회귀 문제입니다.

이 데이터셋에 있는 age와 hours-per-week는 연속형 특성입니다. 그리고 workclass, education, sex, occupation은 범주형 특성입니다. 범위가 아닌 고정된 목록 중에 하나를 값으로 가지며 정성적(qualitative)입니다.

먼저 로지스틱 회귀 분류기를 학습시킵니다. 공식을 다시 서술하면 아래와 같습니다. 

$\hat{y}=w[0] \times x[0] + w[1] \times x[1] + \cdots +w[p] \times x[p] + b > 0$

$w[i]$와 $b$는 훈련 세트로부터 학습되는 계수이고 $x[i]$는 입력 특성입니다. $x[i]$는 숫자여야 하므로 $x[2]$는 'masters'나 'Bachelors'가 될 수 없습니다. 따라서 로지스틱 회귀를 사용하려면 데이터를 다른 방식으로 표현해야 합니다.

## 4.1.1 원-핫-인코딩(가변수)

범주형 변수를 표현하는 데 가장 널리 쓰이는 방법은 '원-핫-인코딩(one-hot encoding)'입니다. '원-아웃-오브-엔 인코딩(one-out-of-N encoding)', '가변수(dummy variable)'로도 불립니다. 가변수는 범주형 변수를 0 또는 1 값을 가진 하나 이상의 새로운 특성으로 바꾼 것입니다. 이렇게 0과 1로 표현된 변수를 선형 이진 분류 공식에(모든 모델에) 적용할 수 있습니다. 

0과 1로 표현된 변수는 개수와 상관없이 범주마다 특성 하나로 표현합니다. 가령 'workclass' 특성에 클래스 네 가지가 존재한다면, 이 클래스 네 가지를 인코딩 하기 위해 새로운 특성 네 가지를 만듭니다. 어떤 사람의 workclass 값에 해당하는 특성은 1이고 나머지 세 특성은 0이 되는 식입니다. 다시 말하면 샘플마다 새로운 특성 하나만 1 값을 갖고 나머지 특성은 0입니다. 원핫, 원아웃오브엔이 의미하는 바입니다.

참고로 원-핫 인코딩은 통계학에서 사용하는 더미 코딩과 유사하지만 차이점이 존재합니다. 머신러닝에서는 특성의 범주를 각기 다른 이진 특성으로 바꾸지만, 통계학에서는 k개의 값을 가진 범주형 특성을 k-1개의 특성으로 변환하는 것이 일반적입니다. 데이터 행렬의 랭크 부족(rank deficient) 현상을 피할 목적입니다. 랭크 부족에 대해서 더 설명하자면, 열 랭크 부족(column rank deficient)은 범주 네 개를 특성 네 개로 인코딩 하면 맨 마지막 특성은 앞의 세 특성을 참조해서 예측할 수 있고, 한 열이 다른 열에 의존적이거나 열의 값이 모두 0인 경우를 뜻합니다. 행렬 분해 방식에 따라서 문제가 될 소지가 있으나 사이킷런에서는 이런 문제를 일으키지 않습니다.

판다스나 사이킷런을 사용해서 범주형 변수를 원-핫-인코딩으로 바꿀 수 있습니다. 먼저 판다스를 사용해서 csv 파일을 읽습니다.

In [21]:
# 이 파일은 열 이름을 나타내는 헤더가 없으므로 `header=None`으로 지정합니다.
# 열 이름을 지정하고자 `names` 매개변수에 리스트 형태의 값을 넣습니다.
data = pd.read_csv(os.path.join(mglearn.datasets.DATA_PATH, 'adult.data'),
                   header=None, index_col=False,
                   names=['age', 'workclass', 'fnlwgt', 'education',
                          'education-num', 'marital-status', 'occupation',
                          'relationship', 'race', 'gender', 'capital-gain',
                          'capital-loss', 'hours-per-week', 'native-country',
                          'income'])
# 지정한 열의 값만 불러옵니다.
data = data[['age', 'workclass', 'education', 'gender', 'hours-per-week',
             'occupation', 'income']]
print(data)
display(data.head())

       age          workclass  ...          occupation  income
0       39          State-gov  ...        Adm-clerical   <=50K
1       50   Self-emp-not-inc  ...     Exec-managerial   <=50K
2       38            Private  ...   Handlers-cleaners   <=50K
3       53            Private  ...   Handlers-cleaners   <=50K
4       28            Private  ...      Prof-specialty   <=50K
...    ...                ...  ...                 ...     ...
32556   27            Private  ...        Tech-support   <=50K
32557   40            Private  ...   Machine-op-inspct    >50K
32558   58            Private  ...        Adm-clerical   <=50K
32559   22            Private  ...        Adm-clerical   <=50K
32560   52       Self-emp-inc  ...     Exec-managerial    >50K

[32561 rows x 7 columns]


,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


### 범주형 데이터 문자열 확인하기

데이터셋을 읽은 후에는 먼저 범주형 데이터를 유심히 살피는 것이 현명합니다. 정해진 범주를 벗어난 값이 있을 수도 있고, 철자나 대소문자가 틀린 값이 있을 수도 있으므로 반드시 데이터를 전처리해야 합니다. 예컨대 남성이 'man'으로 입력되지 않고 'male'로 입력됐다면 이 값들은 필시 같은 범주로 인식하도록 조치해야 합니다. 

열의 내용을 확인하려면 판다스에서 데이터프레임(DataFrame)의 열을 나타내는 Series에 있는 `value_counts` 메서드를 사용합니다. 유일한 값이 몇 번씩 출력되는지 확인할 수 있습니다. 실제로는 모든 열의 값을 살펴야 하지만 여기에서는 gender 열의 값만 확인합니다.

In [22]:
print(data.gender.value_counts())

 Male      21790
 Female    10771
Name: gender, dtype: int64


두 가지 값만을 가지고 있습니다. 원-핫-인코딩으로 나타내기에 적합합니다.

## 4.1.2 숫자로 표현된 범주형 특성

# 4.2 OneHotEncoder와 ColumnTransformer: scikit-learn으로 범주형 변수 다루기

# 4.3 make_column_transformer로 간편하게 ColumnTransformer 만들기

# 4,4 구간 분할, 이산화 그리고 선형 모델, 트리 모델

# 4.5 상호작용과 다항식

# 4.6 일변량 비선형 변환

# 4.7 특성 자동 선택

## 4.7.1 일변량 통계

## 4.7.2 모델 기반 특성 선택

## 4.7.3 반복적 특성 선택

# 4.8 전문가 지식 활용

# 4.9 요약 및 정리